# AI Video Transcriber and Summarizer

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-OpenAI-API-KEY" data-toc-modified-id="1.-OpenAI-API-KEY-1">1. OpenAI API KEY</a></span></li><li><span><a href="#2.-Testing-GPT4-from-LangChain" data-toc-modified-id="2.-Testing-GPT4-from-LangChain-2">2. Testing GPT4 from LangChain</a></span></li><li><span><a href="#3.-Text-Extraction-from-YouTube-with-Whisper" data-toc-modified-id="3.-Text-Extraction-from-YouTube-with-Whisper-3">3. Text Extraction from YouTube with Whisper</a></span></li></ul></div>

## 1. OpenAI API KEY
To carry out this project, we will need an API KEY from OpenAI to use the GPT-4 Turbo model. This API KEY can be obtained at https://platform.openai.com/api-keys. It is only displayed once, so it must be saved at the moment it is obtained. Of course, we will need to create an account to get it.

We store the API KEY in a `.env` file to load it with the dotenv library and use it as an environment variable. This file is added to the `.gitignore` to ensure that it cannot be seen if we upload the code to GitHub, for example.

In [1]:
# import API KEY

import os                           # operating system library
from dotenv import load_dotenv      # load environment variables  


load_dotenv()


OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## 2. Testing GPT4 from LangChain
We are going to test the connection from LangChain to the GPT-4 model. 



In [2]:
from langchain_openai.chat_models import ChatOpenAI   # LangChain connection to OpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4-turbo")

response = model.invoke("What is AI?")

response.content

'AI, or Artificial Intelligence, refers to the simulation of human intelligence in machines that are programmed to think and learn like humans. The term can also apply to any machine that exhibits traits associated with a human mind such as learning and problem-solving.\n\nThe capabilities of AI are vast and can range from performing simple tasks such as recognizing a familiar face or understanding spoken words, to more complex functions like interpreting complex data, driving cars autonomously, or even assisting in advanced fields like medicine, finance, and law.\n\nAI technology can be categorized into two primary types:\n\n1. **Narrow AI**: Also known as Weak AI, this type of artificial intelligence operates within a limited context and is a simulation of human intelligence. Narrow AI is typically focused on performing a single task extremely well and while these machines may seem intelligent, they operate under far more constraints and limitations than even the simplest human intel

## 3. Text Extraction from YouTube with Whisper
First, we need to import the libraries we are going to use. We will use `pytube` to access the video and then `Whisper`, the speech-to-text model from OpenAI. We will also need to install `ffmpeg` according to our operating system so that our machine can analyze the audio coming from the YouTube video.



In [3]:
from pytube import YouTube

import whisper

In [4]:
# url: Ray Kurzweil & Geoff Hinton Debate the Future of AI (29:31)

VIDEO_URL = "https://www.youtube.com/watch?v=kCre83853TM"

In [5]:
# pytube for video extraction

youtube = YouTube(VIDEO_URL)

In [6]:
# we only want the audio from the video

audio = youtube.streams.filter(only_audio=True).first()

audio

<Stream: itag="139" mime_type="audio/mp4" abr="48kbps" acodec="mp4a.40.5" progressive="False" type="audio">

In [8]:
# loading Whisper model in local, 2.88G

whisper_model = whisper.load_model("large-v3")

In [9]:
# whisper model description

whisper_model

Whisper(
  (encoder): AudioEncoder(
    (conv1): Conv1d(128, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
    (blocks): ModuleList(
      (0-31): 32 x ResidualAttentionBlock(
        (attn): MultiHeadAttention(
          (query): Linear(in_features=1280, out_features=1280, bias=True)
          (key): Linear(in_features=1280, out_features=1280, bias=False)
          (value): Linear(in_features=1280, out_features=1280, bias=True)
          (out): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (attn_ln): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Linear(in_features=1280, out_features=5120, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=5120, out_features=1280, bias=True)
        )
        (mlp_ln): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_post): LayerNorm(

**How does Whisper work?**

As you can see, Whisper is essentially divided into two parts: an AudioEncoder and a TextDecoder. This is what is known as an AutoEncoder. The AudioEncoder recognizes the audio and vectorizes it, and from those vectors, the TextDecoder extracts the text. To give you a step-by-step, it would be something like this:

1. Audio Preprocessing: Whisper begins with the preprocessing of the input audio, where the audio quality is adjusted to improve recognition accuracy. This may include normalizing the volume, filtering background noise, and segmenting the audio into more manageable chunks.

2. Feature Extraction: The model extracts features from the processed audio. This involves converting the audio signals into a form that the model can understand, such as spectrograms or Mel-frequency cepstral coefficients (MFCC), which are representations of the sound's frequency and amplitude over time.

3. Learning Model: Whisper uses a neural network model, trained on a large amount of audio data and corresponding transcriptions. This model learns to identify patterns and correlations between the audio and textual transcriptions.

4. Recognition and Translation: During the recognition phase, Whisper converts the audio into text using its neural network. Additionally, it can translate the recognized text into other languages, thanks to its training in multiple languages.

5. Post-processing: Finally, the generated text goes through a post-processing stage to correct common errors, adjust punctuation, and improve the coherence of the text.

We will now use Whisper to transcribe the audio to a text file. First, we'll create a temporary directory where the audio will be saved, and then we'll save Whisper's output in the `transcription.txt` file. If the file already exists, we are not interested in converting it again, since Whisper takes a few minutes to transcribe all the audio, so we only run the model if the text file does not exist.


In [10]:
import tempfile    # handling temporal files


# save path of transcription

PATH_TXT = "../txt/transcription.txt"